# Initial implementation

Following up on the previous notebook, we'll start with the implementation of the Hanoi class:

In [149]:
from typing import Dict, List
from abc import ABC, abstractmethod

class Drawer(ABC):
    @abstractmethod
    def draw(self):
        pass
    
class Solver(ABC):
    @abstractmethod
    def solve(self):
        pass

class Tower(ABC):
    def __init__(self):
        self._discs: List[int] = []
            
    def push(self, disc: int) -> None:
        self._discs.append(disc)
        
    def pop(self) -> int:
        return self._discs.pop()
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs
    
    @abstractmethod
    def draw(self, num_discs: int):
        pass
    
class DefaultTower(Tower):
    def draw(self, num_discs: int):
        empty_level: str = "|"
        tower_base: str = "="

        print(empty_level)
        for disc in range(1, num_discs + 1):
            if disc in self._discs:
                print("X" * disc)
            else:
                print(empty_level)
        print(tower_base * num_discs + "\n")
    
class CircleTower(Tower):
    def draw(self, num_discs: int):
        empty_level: str = "|"
        tower_base: str = "_"

        print(empty_level)
        for disc in range(1, num_discs + 1):
            if disc in self._discs:
                print("O" * disc)
            else:
                print(empty_level)
        print(tower_base * num_discs + "\n")

class AtentionTower(Tower):
    def draw(self, num_discs: int):
        empty_level: str = "!"
        tower_base: str = "$"

        print(empty_level)
        for disc in range(1, num_discs + 1):
            if disc in self._discs:
                print("O" * disc)
            else:
                print(empty_level)
        print(tower_base * num_discs + "\n")

class Hanoi:
    num_discs: int
    source: Tower
    temporary: Tower
    destination: Tower
        
    def __init__(self, num_discs: int):
        self.num_discs = num_discs
        self.source = DefaultTower()
        self.temporary = AtentionTower()
        self.destination = CircleTower()
        
        for i in range(self.num_discs, 0, -1):
            self.source.push(i)   
        
class HanoiDrawer(Drawer):
    @staticmethod
    def draw(hanoi: Hanoi):
        towers: Dict[str, Tower] = {
            "SOURCE": hanoi.source,
            "TEMPORARY": hanoi.temporary,
            "DESTINATION": hanoi.destination,
        }
            
        for name, tower in towers.items():
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            tower.draw(hanoi.num_discs)

class HanoiSolver(Solver):
    def solve(self, source: List[int], destination: List[int], temporary: List[int], num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self.solve(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self.solve(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self.solve(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
            
class HanoiManager:
    def __init__(self, num_discs: int):
        self._hanoi=Hanoi(num_discs)
        self._drawer=HanoiDrawer()
        self._solver=HanoiSolver()
    
    def draw_and_solve(self):
        print("Initial State:\n")
        self._drawer.draw(self._hanoi)
        
        self._solver.solve(source=self._hanoi.source,
            destination=self._hanoi.destination,
            temporary=self._hanoi.temporary,
            num_discs=self._hanoi.num_discs)
        
        print("\n\nFinal State:\n")
        self._drawer.draw(self._hanoi)
        

In [150]:
hanoi = HanoiManager(num_discs=3)
hanoi.draw_and_solve()

Initial State:

SOURCE Tower

|
X
XX
XXX
===

TEMPORARY Tower

!
!
!
!
$$$

DESTINATION Tower

|
|
|
|
___



Final State:

SOURCE Tower

|
|
|
|
===

TEMPORARY Tower

!
!
!
!
$$$

DESTINATION Tower

|
O
OO
OOO
___



# How SOLID is this implementation?

Let's check each of the five principles, and see if this implementation follows them:

## S: Single Responsibility Principle

By quickly inspecting the class, we can see that it's doing at least the following things:

- Creating and populating the towers
- Displaying the towers
- Solving the problem

In short, we're not following the single responsibility principle.

### Exercise - Plan your changes

What changes would you do in order to follow the SRP?

As with most problems in Software Engineering, there's not a single correct answer to this. There's always a tradeoff between how strictly you want to apply the principle and other factors like how much work it will take you to apply it, how easy will it be for new people to understand the code, etc.

We could start by making the following changes:
- Create a Tower class
- Create a HanoiTowers class, that will manage the source/destination/temporary towers
- Create a class to display the towers
- Create a class to solve the problem
- The Hanoi class should be responsible only for "coordinating" the previous.

One could argue that we need even more classes, for instance a ```Disc``` class could make sense.

### Exercise - Make the changes

Now, apply the changes that you planned!

We haven't included the solution here, since it's too large. You can instead find a step-by-step (proposed) solution in this notebook: [SOLUTIONS - Single Responsibility Principle](4.b%20SOLUTIONS%20-%20Single%20Responsibility%20Principle.ipynb)

## O: Open/Closed principle

This one can be a tricky one for this problem. How do we know if we're adhering to this principle?

Here are a few questions that might help you think of it:

- What would you need to do if, for instance, you each tower to be printed differently?

### Exercise

Adapt the code coming from point 2.1 to comply with the Open/Closed principle.

Currently, there's no easy way to *extend* the behaviour for the towers while keeping the *HanoiPrinter* class closed for modification.

So in the [SOLUTIONS - Open/Closed Principle](4.c%20SOLUTIONS%20-%20Open-Closed%20principle.ipynb), we'll be making the necessary changes to comply with the Open/Closed principle.

## L: Liskov's Substituion Principle

Remember what the Liskov's Substitution Principle stated:

```
Functions using base classes must be able to use derived classes without knowing it.
```

### Exercise

Is the current implementation complying with LSP? Why/why not?

Currently, there's little to no inheritance in our project, so we're complying with LSP. There are a few ways that we could be breaking this principle. For instance, imagine you get a requirement for a new type of tower, that you can only use to take discs from, but where you can not put any discs. Translated to code, this means that the *pop* method should work as usual, but the *push* method should raise an error, so you decide to implement a new class, ```PopOnlyTower```. This (far-fetched) example would violate LSP, since we can't just substitute a ```Tower``` instance by a ```PopOnlyTower``` instance.

## I: Interface Segregation Principle

```Many client specific interfaces are better than one general interface```

We don't have "interfaces" in our code, so we're probably good here. Although there's no formal interfaces in Python, you can implement them using abstract classes, which define the methods that child classes need to implement, but with the different that these abstract classes might have code of their own, whereas pure interfaces shouldn't (or in many languages, simply can't) have any code other than the one needed to declare signatures (and sometimes, constants).

We might come back to this principle after we work on the Dependency Inversion Principle.

## D: Dependency Inversion Principle

```Classes should depend on an abstraction rather than on a concretion```

What does it actually mean for us? Well, most of our classes depend on other classes. However, all of these dependencies are hardly coupled (they depend on the exact classes). This increases the risk that you start to "know" too much of your dependency, increasing coupling.

Instead, if your dependency is against an abstraction (i.e. an interface), all your class needs to know is "the contract" it should adhere to (and the "contract" it can expect from the other side).

### Exercise

How would you apply the Dependency Inversion Principle to the code we have?

Check [SOLUTIONS - Dependency Inversion Principle](4.d%20SOLUTIONS%20-%20Dependency%20Inversion%20Principle.ipynb#) for a discussion on changes we might want to do for this principle

# Final thoughts

If you reached this point, congratulations!

In case you wonder, the code we have shown through this course is definitely not perfect, and there are still lots of things that could be improved. The main goal was to introduce the different concepts in a hands-on way, while emphasizing that often there's a trade-off that you will need to balance between staying theoretically correct and staying pragmatic.

At a first glance, anyone a bit experienced with the SOLID principles would tell you, for instance, that the Tower class we have is not compliant with the principles. Although it knows *nothing* about other classes, the fact that we have the characters we use to print the tower in the Tower class itself is (indirectly) keeping the class coupled with how it is displayed in a specific medium (the console). Imagine, what would happen if you need to represent the towers also in HTML, JSON, etc?